###Importamos las librerias que nos seran utiles

In [90]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import re

#Importamos las funciones que nos serán de utilidad

#Importamos los datos

In [91]:
sharks = pd.read_csv('/home/julian/Cursos/Ironhack/Proyecto1/shark-data-cleaning/attacks.csv', encoding="ISO-8859-1")

In [180]:
sdf = pd.DataFrame(sharks)

In [181]:
sdf.dtypes

Case Number                object
Date                       object
Year                      float64
Type                       object
Country                    object
Area                       object
Location                   object
Activity                   object
Name                       object
Sex                        object
Age                        object
Injury                     object
Fatal (Y/N)                object
Time                       object
Species                    object
Investigator or Source     object
pdf                        object
href formula               object
href                       object
Case Number.1              object
Case Number.2              object
original order            float64
Unnamed: 22                object
Unnamed: 23                object
dtype: object

##Como podemos observar, la mayoria de los datos son no numericos.

### LIMPIEZA HIPOTESIS I

##Vamos a eliminar las columnas que no son relevantes para el analisis de la primer hipotesis

In [182]:
def drop():
    relevant = ['Case Number', 'Type', 'Case Number.1', 'Case Number.2']
    for i in sdf.columns:
        if i not in relevant:
            sdf.drop(columns=[i], inplace=True)

In [183]:
drop()

#Las columnas 'Case Number', 'Case Number.1' y 'Case Number.2' son relevantes para el analisis, sin embargo, vamos a chequear en que medida los valores de las columnas se parecen para prescindir de alguna de las tres ##DEFINIR FUNCION

In [192]:
def compare(): 
        compare1 = np.where(sdf['Case Number'] == sdf['Case Number.1'], True, False)
        sdf['equal1'] = compare1
        return print(sdf['equal1'].value_counts())
        compare2 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal2'] = compare2
        return print(sdf['equal2'].value_counts())
        compare3 = np.where(sdf['Case Number'] == sdf['Case Number.2'], True, False)
        sdf['equal3'] = compare3
        return print(sdf['equal3'].value_counts())
        
            
            

In [193]:
compare()

False    19445
True      6278
Name: equal1, dtype: int64


In [94]:
compare = np.where(sdf['Case Number.1'] == sdf['Case Number.2'], True, False)

In [95]:
sdf['equal'] = compare

In [96]:
sdf['equal'].value_counts() 

False    19441
True      6282
Name: equal, dtype: int64

In [97]:
sdf.isnull().sum()

Case Number      17021
Type             19425
Case Number.1    19421
Case Number.2    19421
equal                0
dtype: int64

##SI COMPARAMOS LAS TRES COL ENTRE SI, EN DONDE MENOS DIF DE VAL HAY ES ENTRE CAS NBR Y CAS NBR.2, AUNQUE LA DI ES MINIMA. SIN EMBARGO DE LAS TRES COL LA QUE MENOS CANT DE NULL TIENE ES CASE NBR, NOS QUEDAMOS CON ELLA##DEJAMOS SOLO LAS COLUMNAS CASE NR Y TYPE

In [98]:
sdf.drop(columns=['Case Number.1', 'Case Number.2', 'equal'], inplace=True)

,Case Number,Type
0,2018.06.25,Boating
1,2018.06.18,Unprovoked
2,2018.06.09,Invalid
3,2018.06.08,Unprovoked
4,2018.06.04,Provoked


##VAMOS A ELIMINAR LOS VALORES QUE NO SEAN FECHA, YA QUE NOS INTERESA VER LA ESTACIONALIDAD. Empezamos por los valores NaN, y luego añadiremos una columna solo con los valores que contengan informacion sobre el mes, ya que nos interesa solo la indfo mensual

In [99]:
sdf.dropna(inplace=True)

In [100]:
def filtracion(x):
    months = {'01':'Jan','02':'Feb','03':'Mar','04':'Apr','05':'May','06':'Jun','07':'Jul','08':'Aug','09':'Sep','10':'Oct','11':'Nov','12':'Dec'}
    m = re.findall('\d{4}.\d{2}.\d{2}', x)
    if m and m[0][5:7] != '00':
        for k, v in months.items():
            if k == m[0][5:7]:
                return v

In [101]:
sdf['Months'] = sdf['Case Number'].apply(filtracion)

##Borramos los valores nulos para los datos que no tenian registro mensual

In [102]:
sdf.dropna(inplace=True)

In [147]:
sdf.head()

,Case Number,Type,Months
0,2018.06.25,Boating,Jun
1,2018.06.18,Unprovoked,Jun
2,2018.06.09,Invalid,Jun
3,2018.06.08,Unprovoked,Jun
4,2018.06.04,Provoked,Jun


##aHORA CONSTRUIMOS UN DATAFRAME CUYAS COLUMNAS CONTENDRAN LAS FRECUENCIAS MENSUALES DE ATAQUES Y LOS MESES CORRESPNDIENTES consideramos todos los tipos  de ataque.

In [142]:
ataques_mensuales = pd.DataFrame(sdf['Months'].value_counts()).reset_index()

##RENOMBRAMOS LAS COLUMNAS PARA EMPROLIJAR

In [155]:
c = {'Months': 'Monthly Attacks', 'index':'Month'}
ataques_mensuales = ataques_mensuales.rename(columns=c)

In [156]:
at = ataques_mensuales.reindex([3, 11, 8, 5, 10, 4, 0, 1, 2, 7, 9, 6])